In [235]:
from __future__ import division
from math import log, exp, factorial, ceil
from random import random
from collections import Counter, defaultdict
import numpy as np
COUNT = 0

In [1]:
LFMEMO = {}
def logfact(n):
    # return ln(n!)
    # TODO: make smarter
    if n in LFMEMO:
        return LFMEMO[n]
    lf = sum(log(i) for i in range(1, n+1))
    LFMEMO[n] = lf
    return lf

LTMEMO = {}
def logtutte(n, m):
    global COUNT
    COUNT = COUNT + 1
    if n < 0 or m < 0:
        return -float('inf')
    #if (n, m) in LTMEMO:
        #return LTMEMO[(n, m)]
    val = (n+1)*log(2) + logfact(2*m+1) + logfact(2*m+3*n)
    val -= logfact(m)*2 + logfact(n) + logfact(2*m+2*n+2)    
    #LTMEMO[(n, m)] = val
    return val

def tutte(n, m):
    return round(exp(logtutte(n, m)))

In [2]:
def rtutte(n, m):
    total = 0
    if n > 0:
        val = tutte(n-1, m+1)
        print('inner', val)
        total += val
    for j in range(0, m):
        for k in range(0, n+1):
            a = tutte(k, j)
            b = tutte(n-k, m-j-1)
            print((k, j), (n-k, m-j-1), a, b, a*b)
            total += a*b
    return total

In [3]:
def logsample(N):
    probs = []
    sizes = []
    for n in range(0, N-1):
        m = N - n - 2
        probs.append(logtutte(n, m))
        sizes.append((n, m))
    high = max(probs)
    probs = [exp(p - high) for p in probs]
    total = sum(probs)
    probs = [p/total for p in probs]
    tot = 0
    for i in range(len(probs)):
        tot += probs[i]
    n, m = sizes[np.random.choice(N-1, p=probs)]
    return n, m

In [239]:
def choose(n, m, logremaining_graphs):
    r = random()
    total = 0.0
    for j in range(0, m):
        for k in range(0, n+1):
            logcount = logtutte(k, j) + logtutte(n-k, m-j-1)
            p = exp(logcount - logremaining_graphs)
            total += p
            if total >= r:
                return k, j
            # there are [count] triangulations with k internal and j+2 external vertices on one side and the rest on the other
    raise ValueError("should have returned something...%s"%total)

# better method for choosing k, j
def choose2(n, m, logremaining_graphs):
    r = random()
    total = 0.0
    if m >= (n+1):
        for d in range(n+1):
            count = ((d+1)*m)//(n+1)
            for c in range(count):
                k, j = (d - int(ceil((n+1)*(c+1)/m)) + 1, c)
                amt = logtutte(k, j) + logtutte(n-k, m-j-1)
                amt = exp(amt - logremaining_graphs)
                total += 2*amt
                if n == n-k and m == m-j-1:
                    total -= amt
                if total >= r:
                    if random() < 0.5:
                        return k, j
                    else:
                        return n-k, m-j-1
    else:
        for d in range(m):
            count = ((d+1)*(n+1))//m
            for c in range(count):
                k, j = (c, d - int(ceil(m*(c+1)/(n+1))) + 1)
                amt = logtutte(k, j) + logtutte(n-k, m-j-1)
                amt = exp(amt - logremaining_graphs)
                total += 2*amt
                if n == n-k and m == m-j-1:
                    total -= amt
                if total >= r:
                    if random() < 0.5:
                        return k, j
                    else:
                        return n-k, m-j-1
    raise ValueError("should have returned something...%s"%total)

def logmake_graph(G, internal, external):
    n = len(internal)
    m = len(external)-2
    if n == 0 and m == 0:
        # nothing else to add
        return
    logtotal_graphs = logtutte(n, m)
    logadd_internal = logtutte(n-1, m+1)
    if random() < exp(logadd_internal - logtotal_graphs):
        # add internal triangle
        triangle = (external[0], external[1], internal[0])
        G.append(triangle)
        logmake_graph(G, internal[1:], external + [internal[0]])
    else:
        logremaining_graphs = logtotal_graphs + log(1 - exp(logadd_internal - logtotal_graphs))
        k, j = choose2(n, m, logremaining_graphs)
        triangle = (external[0], external[1], external[j+2])
        G.append(triangle)
        left = external[1:j+3]
        right = external[j+2:] + [external[0]]
        logmake_graph(G, internal[:k], left)
        logmake_graph(G, internal[k:], right)
    return G

def sample_graph(n, m):
    internal = range(m+2, m+n+2)
    external = range(0, m+2)
    G = []
    return logmake_graph(G, internal, external)

In [ ]:
# show that the number of graphs generated by this algorithm is the same(ish) as T(n, m)
from collections import Counter
graphs = Counter()
for _ in range(100000):
    n = 4
    m = 2
    internal = range(m+2, m+n+2)
    external = range(0, m+2)
    G = []
    graphs[tuple(logmake_graph(G, internal, external))] += 1
print len(graphs), exp(logtutte(n, m))
#print graphs

In [ ]:
# estimating runtime -- assumed n^1.5
COUNT = 0
SIZE = 2560
for _ in range(20):
    G = []
    sample_graph(*logsample(SIZE))
print COUNT/20

In [28]:
import time
start = time.time()
for _ in range(50):
    x = sample_graph(*logsample(200))
print time.time() - start

13.8935880661


In [ ]:
# lol this doesn't really work
def make_dual(g):
    triples = set()
    names = []
    pairs = defaultdict(set)
    for triple in g:
        name = str(triple)
        if triple in triples:
            name += "'"
        triples.add(triple)
        triple = sorted(triple)
        pairs[tuple((triple[0], triple[1]))].add(name)
        pairs[tuple((triple[1], triple[2]))].add(name)
        pairs[tuple((triple[0], triple[2]))].add(name)
        names.append(name)
    for p in pairs:
        if len(pairs[p]) > 1:
            print pairs[p]

make_dual(x)

In [247]:
import sys
sys.setrecursionlimit(5000)
x = sample_graph(*logsample(10000))

In [252]:
import cProfile
cProfile.run('sample_graph(*logsample(2000))')

         1522205 function calls (1516212 primitive calls) in 0.750 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   185695    0.338    0.000    0.561    0.000 <ipython-input-236-900dbb94d879>:12(logtutte)
   924920    0.189    0.000    0.189    0.000 <ipython-input-236-900dbb94d879>:2(logfact)
        1    0.005    0.005    0.022    0.022 <ipython-input-238-1ff0ae31bde7>:1(logsample)
     1998    0.128    0.000    0.673    0.000 <ipython-input-239-d5a003a618f7>:14(choose2)
   5994/1    0.030    0.000    0.728    0.728 <ipython-input-239-d5a003a618f7>:45(logmake_graph)
        1    0.000    0.000    0.728    0.728 <ipython-input-239-d5a003a618f7>:69(sample_graph)
        1    0.000    0.000    0.750    0.750 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 getlimits.py:94(__new__)
    11989    0.001    0.000    0.001    0.000 {len}
    87853    0.008    0.000    0.008    0.000 {math.ceil}
    95845    0.0